<a href="https://colab.research.google.com/github/mildsupitcha25/MachineLearning_phishing_email/blob/main/ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import xgboost as xgb

## --- 1. การสร้างข้อมูลจำลอง (Simulated Stylometric Data) ---
# ในสถานการณ์จริง คุณจะต้องใช้ข้อมูล stylometric 60 ฟีเจอร์ที่สกัดจากอีเมล
# เช่น imperative_verbs_count, clause_density, first_person_pronoun_count
# โดยมีจำนวนตัวอย่างทั้งหมด 126 อีเมล (63 Phishing, 63 Legitimate)

n_samples = 126 # จำนวนตัวอย่างทั้งหมดในงานวิจัย (63 Phishing + 63 Legitimate) [cite: 143]
n_features = 60 # จำนวน Stylometric Features ที่ใช้ [cite: 304, 316]

# สร้าง DataFrame จำลอง (สมมติว่า 'X' คือ 60 ฟีเจอร์ที่ผ่านการสกัดแล้ว)
# สร้างข้อมูลที่มีความแตกต่างกันเล็กน้อยระหว่าง 2 Class เพื่อให้โมเดลสามารถเรียนรู้ได้
X = np.random.rand(n_samples, n_features) * 5
y = np.array([0] * (n_samples // 2) + [1] * (n_samples - n_samples // 2)) # 0: Legitimate, 1: Phishing

# ปรับให้ข้อมูล Phishing (y=1) มีค่าในฟีเจอร์สำคัญสูงขึ้นเล็กน้อย
# (เช่น ฟีเจอร์ 0 สมมติเป็น imperative_verbs_count)
X[y == 1, 0] = X[y == 1, 0] + 1.5
# (ฟีเจอร์ 1 สมมติเป็น clause_density)
X[y == 1, 1] = X[y == 1, 1] + 1.0


## --- 2. การเตรียมข้อมูล (Data Preprocessing) ---

# 2.1. การปรับมาตรฐาน (Feature Scaling)
# StandardScaler จะแปลงข้อมูลให้มีค่าเฉลี่ยเป็น 0 และส่วนเบี่ยงเบนมาตรฐานเป็น 1
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2.2. การแบ่งชุดข้อมูล (Train-Test Split)
# แบ่งเป็น Training 80% และ Testing 20%
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.20, random_state=42, stratify=y)
# stratify=y ช่วยให้สัดส่วนของ Phishing/Legitimate ในชุด Train/Test ใกล้เคียงกัน


## --- 3. การฝึกโมเดล XGBoost (XGBoost Model Training) ---

# 3.1. กำหนดค่า Hyperparameters
# ใช้ค่าเริ่มต้นจาก Scikit-learn และตามที่งานวิจัยระบุ (eval_metric='logloss', random_state=42)
xgb_model = xgb.XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

# 3.2. ฝึกโมเดล
print(f"กำลังฝึก XGBoost บนข้อมูล {len(X_train)} ตัวอย่าง...")
xgb_model.fit(X_train, y_train)
print("ฝึกโมเดลเสร็จสิ้น")


## --- 4. การประเมินผล (Model Evaluation) ---

# 4.1. การทำนาย (Prediction)
y_pred = xgb_model.predict(X_test)
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1] # สำหรับคำนวณ AUC

# 4.2. คำนวณเมตริกการประเมินผล
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_score = roc_auc_score(y_test, y_pred_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel() # ดึงค่าจาก Confusion Matrix

# 4.3. แสดงผลลัพธ์
print("\n" + "="*50)
print("✨ ผลลัพธ์การประเมินโมเดล XGBoost บนชุดทดสอบ 20% ✨")
print("="*50)
print(f"Accuracy: {accuracy:.4f} (งานวิจัยทำได้ 0.96) [cite: 576]")
print(f"Precision: {precision:.4f} (งานวิจัยทำได้ 0.96) [cite: 601]")
print(f"Recall: {recall:.4f} (งานวิจัยทำได้ 0.96) [cite: 601]")
print(f"F1-Score: {f1:.4f} (งานวิจัยทำได้ 0.96) [cite: 601]")
print(f"AUC-Score: {auc_score:.4f} (งานวิจัยทำได้ 0.99) [cite: 601]")

print("\n--- Confusion Matrix ---")
print(f"True Negatives (TN): {tn}") # Legitimate ถูกทำนายถูก
print(f"False Positives (FP): {fp}") # Legitimate ถูกทำนายผิดเป็น Phishing
print(f"False Negatives (FN): {fn}") # Phishing ถูกทำนายผิดเป็น Legitimate
print(f"True Positives (TP): {tp}") # Phishing ถูกทำนายถูก [cite: 575]

กำลังฝึก XGBoost บนข้อมูล 100 ตัวอย่าง...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:09:19] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


ฝึกโมเดลเสร็จสิ้น

✨ ผลลัพธ์การประเมินโมเดล XGBoost บนชุดทดสอบ 20% ✨
Accuracy: 0.6923 (งานวิจัยทำได้ 0.96) [cite: 576]
Precision: 0.6667 (งานวิจัยทำได้ 0.96) [cite: 601]
Recall: 0.7692 (งานวิจัยทำได้ 0.96) [cite: 601]
F1-Score: 0.7143 (งานวิจัยทำได้ 0.96) [cite: 601]
AUC-Score: 0.7515 (งานวิจัยทำได้ 0.99) [cite: 601]

--- Confusion Matrix ---
True Negatives (TN): 8
False Positives (FP): 5
False Negatives (FN): 3
True Positives (TP): 10


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from catboost import CatBoostClassifier


# กำหนดชื่อไฟล์ CSV ของคุณ
CSV_FILE = 'stylometric_features.csv'

# 2.1. โหลดข้อมูล
try:
    data = pd.read_csv(CSV_FILE)
    print(f"โหลดข้อมูลจาก {CSV_FILE} สำเร็จ. ขนาดข้อมูล: {data.shape}")
except FileNotFoundError:
    print(f"เกิดข้อผิดพลาด: ไม่พบไฟล์ {CSV_FILE} โปรดตรวจสอบชื่อไฟล์")
    exit()

# 2.2. แยกฟีเจอร์ (X) และป้ายกำกับ (y)
# 'Label' คือชื่อคอลัมน์ของผลลัพธ์ (เป้าหมาย)
X = data.drop('Label', axis=1)
y = data['Label']

# 2.3. การปรับมาตรฐาน (Feature Scaling)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns) # แปลงกลับเป็น DataFrame เพื่อให้จัดการง่าย

# 2.4. การแบ่งชุดข้อมูล (Train-Test Split)
# แบ่งเป็น Training 80% และ Testing 20%
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.20, random_state=42, stratify=y)

print(f"ขนาดชุดฝึก (Train Set): {len(X_train)} ตัวอย่าง")
print(f"ขนาดชุดทดสอบ (Test Set): {len(X_test)} ตัวอย่าง")

# --- 3. การฝึกโมเดล CatBoost และการประเมินผล ---

# 3.1. กำหนดค่า Hyperparameters
cb_model = CatBoostClassifier(
    iterations=100,
    loss_function='Logloss',
    random_state=42,
    verbose=0,
    early_stopping_rounds=10
)

# 3.2. ฝึกโมเดล
print("\n" + "="*50)
print("กำลังฝึก CatBoost Model...")
cb_model.fit(X_train, y_train)
print("ฝึกโมเดลเสร็จสิ้น")
print("="*50)

# 3.3. การประเมินผล
y_pred = cb_model.predict(X_test)
y_pred_proba = cb_model.predict_proba(X_test)[:, 1]

# คำนวณเมตริก
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_score = roc_auc_score(y_test, y_pred_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# 3.4. แสดงผลลัพธ์
print("✨ ผลลัพธ์การประเมินโมเดล CatBoost บนชุดทดสอบ 20% ✨")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"AUC-Score: {auc_score:.4f}")

print("\n--- Confusion Matrix ---")
print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")
print(f"True Positives (TP): {tp}")

ModuleNotFoundError: No module named 'catboost'

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# --- 1. การสร้างข้อมูลจำลอง (Simulated Data) ---
# ในสถานการณ์จริง คุณจะโหลดข้อมูลอีเมลของคุณเอง
data = {
    'text': [
        "Congratulations! You've won a free iPhone. Click here now.",
        "Meeting agenda attached, please review before tomorrow's session.",
        "URGENT: Your account has been suspended. Verify your details immediately.",
        "Can we reschedule our lunch meeting to next week?",
        "Viagra and Cialis at discount prices! Limited stock.",
        "Your order #1001 has been shipped and is on its way.",
        "Hello, this is a test email for spam filtering.",
        "Claim your prize money! Send us your bank details fast.",
        "Don't miss out on this exclusive offer, 50% off everything!",
        "Regarding the project proposal, I've made some revisions."
    ],
    'label': [
        'spam', 'ham', 'spam', 'ham', 'spam',
        'ham', 'ham', 'spam', 'spam', 'ham'
    ]
}

df = pd.DataFrame(data)

# แปลง Label ให้อยู่ในรูปตัวเลข (0=ham, 1=spam)
df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})
X = df['text']
y = df['label_num']


# --- 2. การแบ่งข้อมูล (Train-Test Split) ---
# แบ่งข้อมูลเป็นชุดฝึก (Training) และชุดทดสอบ (Testing)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y)
# stratify=y ช่วยรักษาสัดส่วน spam/ham ในชุดข้อมูล


# --- 3. การสร้างคุณลักษณะ (Feature Engineering: TF-IDF) ---

# TfidfVectorizer แปลงข้อความให้เป็นเวกเตอร์ของตัวเลข โดยให้น้ำหนักคำที่สำคัญ
vectorizer = TfidfVectorizer(stop_words='english', lowercase=True)

# ฝึก Vectorizer ด้วยข้อมูลชุดฝึก และแปลงชุดฝึก
X_train_vec = vectorizer.fit_transform(X_train)

# แปลงชุดทดสอบ (ใช้ fit ที่ฝึกจากชุดฝึกเท่านั้น เพื่อป้องกัน Data Leakage)
X_test_vec = vectorizer.transform(X_test)


# --- 4. การฝึกโมเดล (Model Training: Multinomial Naïve Bayes) ---

# MultinomialNB เหมาะสำหรับคุณลักษณะที่เป็นการนับ (Count) หรือความถี่ (Frequency)
nb_model = MultinomialNB()

# ฝึกโมเดล
nb_model.fit(X_train_vec, y_train)


# --- 5. การประเมินผล (Model Evaluation) ---

# ทำนายผลบนชุดทดสอบ
y_pred = nb_model.predict(X_test_vec)

# คำนวณความแม่นยำและเมตริกอื่น ๆ
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred, target_names=['Ham (0)', 'Spam (1)'])

print("="*50)
print("✨ ผลลัพธ์การจำแนก Spam/Ham ด้วย Naïve Bayes ✨")
print("="*50)
print(f"ความแม่นยำ (Accuracy): {accuracy:.4f}")
print("\n--- Confusion Matrix ---")
print(conf_matrix)
print("\n--- Classification Report ---")
print(class_report)

# --- 6. การทดสอบกับอีเมลใหม่ (New Prediction) ---
new_email = ["Verify your account password now or it will be deleted.",
             "Just checking in on the report status, no rush."]
new_email_vec = vectorizer.transform(new_email)
new_pred = nb_model.predict(new_email_vec)

print("\n--- การทำนายอีเมลใหม่ ---")
for i, pred in enumerate(new_pred):
    label = 'SPAM' if pred == 1 else 'HAM'
    print(f"อีเมล: '{new_email[i]}' -> ทำนายเป็น: {label}")

✨ ผลลัพธ์การจำแนก Spam/Ham ด้วย Naïve Bayes ✨
ความแม่นยำ (Accuracy): 0.6667

--- Confusion Matrix ---
[[1 1]
 [0 1]]

--- Classification Report ---
              precision    recall  f1-score   support

     Ham (0)       1.00      0.50      0.67         2
    Spam (1)       0.50      1.00      0.67         1

    accuracy                           0.67         3
   macro avg       0.75      0.75      0.67         3
weighted avg       0.83      0.67      0.67         3


--- การทำนายอีเมลใหม่ ---
อีเมล: 'Verify your account password now or it will be deleted.' -> ทำนายเป็น: SPAM
อีเมล: 'Just checking in on the report status, no rush.' -> ทำนายเป็น: SPAM
